<a href="https://colab.research.google.com/github/izmare/Flux_Dev_i2i/blob/main/FluxDevi2i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup TotoroUI and Download Models
import os
import subprocess
from IPython.display import display, clear_output

# Change directory to /content
%cd /content

# Clone the repository if it hasn't been done yet
if not os.path.exists('/content/TotoroUI'):
    !git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI

# Change directory to /content/TotoroUI
%cd /content/TotoroUI

# Install necessary packages
!pip install -q torchsde einops diffusers accelerate xformers==0.0.27.post2
!apt -y install -qq aria2

# Model download options
download_flux_dev = True  #@param {type:"boolean"}
download_flux_schnell = False  #@param {type:"boolean"}

# Download the FLUX.1-dev model
if download_flux_dev:
    if not os.path.exists('/content/TotoroUI/models/unet/flux1-dev-fp8.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
    if not os.path.exists('/content/TotoroUI/models/vae/ae.sft'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
    if not os.path.exists('/content/TotoroUI/models/clip/clip_l.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
    if not os.path.exists('/content/TotoroUI/models/clip/t5xxl_fp8_e4m3fn.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

# Download the FLUX.1-schnell model
if download_flux_schnell:
    if not os.path.exists('/content/TotoroUI/models/unet/flux1-schnell-fp8.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-schnell/resolve/main/flux1-schnell-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-schnell-fp8.safetensors
    if not os.path.exists('/content/TotoroUI/models/vae/ae_schnell.sft'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-schnell/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae_schnell.sft
    if not os.path.exists('/content/TotoroUI/models/clip/clip_l_schnell.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-schnell/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l_schnell.safetensors
    if not os.path.exists('/content/TotoroUI/models/clip/t5xxl_fp8_schnell.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-schnell/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_schnell.safetensors

# Display loading bar for progress
from tqdm.notebook import tqdm
import time

for i in tqdm(range(100), desc="Loading Models..."):
    time.sleep(0.05)
clear_output()

# Import necessary modules
import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_post_processing
from totoro import model_management

# Initialize model components
DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
VAEEncode = NODE_CLASS_MAPPINGS["VAEEncode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
ImageScaleToTotalPixels = nodes_post_processing.NODE_CLASS_MAPPINGS["ImageScaleToTotalPixels"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

# Function to convert input image to PNG if not already
def convert_to_png(image_path):
    img = Image.open(image_path)
    if img.format != 'PNG':
        png_path = os.path.splitext(image_path)[0] + '.png'
        img.save(png_path)
        return png_path
    return image_path

# Function to generate an incremental file name
def generate_incremental_filename(directory, base_name='output', ext='.png'):
    i = 0
    while os.path.exists(f"{directory}/{base_name}_{i:04d}{ext}"):
        i += 1
    return f"{directory}/{base_name}_{i:04d}{ext}"

In [ ]:
#@title Image Generation with Guidance Scale and Conversion to PNG
import torch
import random
import string
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm

# Input fields
positive_prompt = "A serene mountain landscape"  #@param {type:"string"}
width = 512  #@param {type:"slider", min:0, max:2048, step:64}
height = 1024  #@param {type:"slider", min:0, max:2048, step:64}
seed = 0  #@param {type:"number"}
steps = 20  #@param {type:"slider", min:10, max:100, step:5}
sampler_name = "euler"  #@param ["euler", "ddim", "plms", "heun"]
scheduler = "simple"  #@param ["simple", "linear", "cosine"]
guidance_scale = 3.5  #@param {type:"slider", min:1.0, max:10.0, step:0.1}
image_dir = "/content/test.png"  #@param {type:"string"}

# Convert image to PNG if necessary
image_path = convert_to_png(image_dir)

# Generate random seed if 0
if seed == 0:
    seed = random.randint(1, 1e6)
torch.manual_seed(seed)

# Initialize latent image
latent_image = EmptyLatentImage.new_latent(width, height)

# Create and configure noise generator
random_noise = RandomNoise.generate_noise(width, height, steps, scheduler, seed)

# Image generation process
with torch.no_grad():
    for i in tqdm(range(steps), desc="Generating Image..."):
        latent_image = unet(latent_image, random_noise[i])
        latent_image = BasicGuider.apply_guidance_scale(latent_image, guidance_scale)
        latent_image = KSamplerSelect.select(latent_image, sampler_name)

# Decode latent image to RGB
decoded_image = VAEDecode.decode(latent_image)

# Scale image to desired size
scaled_image = ImageScaleToTotalPixels.scale(decoded_image, width * height)

# Save output image with incremental file name
output_path = generate_incremental_filename("/content", "output")
scaled_image.save(output_path)

# Display the generated image
display(Image.open(output_path))
print(f"Image saved at {output_path}")